In [ ]:
!nvidia-smi 

Thu Aug  5 05:45:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**1. Setup conda enviroment**

##**1.1 Tải và cài đặt conda**

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

##**1.2. cập nhật các gói  và cài đặt tensorflow**

In [ ]:
!rm Miniconda3-4.5.4-Linux-x86_64.sh
!conda install --channel defaults conda python=3.9.5 --yes
!conda update --channel defaults --all --yes
!conda update --channel conda-forge --all --yes
!conda install ipykernel --yes
!conda install -c fastchan numpy=1.19.5 --yes
!pip install Cython

In [ ]:
!python --version

Python 3.9.6


#**2. Chuẩn bị các công cụ cần thiết**

##**2.1. Dataset**

In [ ]:
# Tải dataset từ Drive
!gdown --id 1G6n6en3PnCCqGuuEEIQIcGa5I9Yk6mI6

##**2.2. Protoc**

In [ ]:
# Tải gói protoc
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.17.3/protoc-3.17.3-linux-x86_64.zip

In [ ]:
# Giải nén gói protoc vừa tải
!unzip protoc-3.17.3-linux-x86_64.zip -d protoc

##**2.3. Pre-trainded-model từ tensorflow**

In [ ]:
!git clone https://github.com/tensorflow/models.git

##**2.4. Protobuf Installation/Compilation**

In [ ]:
%cd models/research/
!/content/protoc/bin/protoc object_detection/protos/*.proto --python_out=.

##**2.5. Cài đặt COCO API**

In [ ]:
%cd /content/

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
!make --directory=cocoapi/PythonAPI/
!cp -r cocoapi/PythonAPI/pycocotools models/research/

#**3. Setup model**

##**3.1. Tạo các thư mục cần thiết**

In [ ]:
!mkdir -p training
!mkdir -p training/annotations
!mkdir -p training/exproted-models
!mkdir -p training/images
!mkdir -p training/models
!mkdir -p training/pre-trained-models

##**3.2. Chuẩn bị dataset**

In [ ]:
!unzip dataset.zip
!mv train training/images
!mv test training/images

#**4. Chuẩn bị các file cần cho quá trình huấn luyện**

In [ ]:
%cd /content/training

##**4.1. label_map.pbtxt**

In [ ]:
!touch annotations/label_map.pbtxt

In [ ]:
%%writefile annotations/label_map.pbtxt
item {
    id: 1
    name: 'correct'
    display_name: 'true'
}

item {
    id: 2
    name: 'incorrect'
    display_name: 'wrong'
}

item {
    id: 3
    name: 'face'
    display_name: 'face'
}


##**4.2. generate_tfrecord.py và ssd_resnet50_v1_fpn**

In [ ]:
# Tải file generate_tfrecord.py
!wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py

In [ ]:
# Tải file ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

# Giải nén Tải file ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz -C pre-trained-models/
!rm ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

##**4.3. Cài đặt Object Detection API**

In [ ]:
%cd /content/

In [ ]:
!cp models/research/object_detection/packages/tf2/setup.py models/research/.
!python -m pip install --use-feature=2020-resolver models/research/.

##**4.4. train.record và test.record**

In [ ]:
%cd /content/training

In [ ]:
# Tạo file train.record
!python generate_tfrecord.py \
        -x images/train \
        -l annotations/label_map.pbtxt \
        -o annotations/train.record

# Tạo file test.record
!python generate_tfrecord.py \
        -x images/test \
        -l annotations/label_map.pbtxt \
        -o annotations/test.record

##**4.4. pipeline.config**

In [ ]:
!mkdir -p models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

!cp pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config \
    models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/

!cp ../models/research/object_detection/model_main_tf2.py ./

In [ ]:
%%writefile /content/training/models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_resnet50_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.00039999998989515007
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 0.00039999998989515007
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.996999979019165
            scale: true
            epsilon: 0.0010000000474974513
          }
        }
        depth: 256
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 9.99999993922529e-09
        iou_threshold: 0.6000000238418579
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 4
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.03999999910593033
          total_steps: 25000
          warmup_learning_rate: 0.013333000242710114
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "models/checkpoint/ckpt-16"
  num_steps: 25000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: true
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "annotations/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "annotations/train.record"
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "annotations/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "annotations/test.record"
  }
}


#**5. Train model**

##**5.1. Kiểm tra GPU**

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices))

##**5.2. Training**

###**5.2.1. Xử lý xung đột gói**

In [ ]:
%cd /content/

In [ ]:
!conda install -c fastchan python=3.8.5 --yes
!pip install tensorflow==2.3.0 tensorflow-gpu==2.3.0

In [ ]:
!python -m pip install --use-feature=2020-resolver models/research/.

In [ ]:
!conda install -c fastchan python=3.8.5 --yes
!pip install tensorflow==2.3.0 tensorflow-gpu==2.3.0

###**5.2.2. Train**

In [ ]:
%cd /content/training

In [ ]:
!conda install ipykernel --yes

In [ ]:
!python model_main_tf2.py \
        --model_dir=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 \
        --pipeline_config_path=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config

2021-08-05 07:28:55.961056: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-05 07:29:37.705237: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-05 07:29:37.883414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-05 07:29:37.888099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P4 computeCapability: 6.1
coreClock: 1.1135GHz coreCount: 20 deviceMemorySize: 7.43GiB deviceMemoryBandwidth: 178.99GiB/s
2021-08-05 07:29:37.888482: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-05 07:29:38.773859: I tensorflow/stream_executor/platform/defaul

##**5.3. Lưu model**

###**5.3.1. Xuất model vừa train vào thư mục exported-models**

In [ ]:
!cp /content/models/research/object_detection/exporter_main_v2.py \
    /content/training/

In [ ]:
!python exporter_main_v2.py \
        --input_type image_tensor \
        --pipeline_config_path models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config \
        --trained_checkpoint_dir models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ \
        --output_directory exported-models/

###**5.3.2. Lưu model vào google drive**

In [ ]:
%cd /content/
!mkdir drive/MyDrive/ssd_resnet50/
!cp -vr training/ drive/MyDrive/ssd_resnet50/

In [ ]:
!cp -vr training/exported-models drive/MyDrive/ssd_resnet50/

In [ ]:
!zip -vr training.zip training/

In [ ]:
!cp -vf training.zip drive/MyDrive/ssd_resnet50/

'training.zip' -> 'drive/MyDrive/ssd_resnet50/training.zip'
